In [ ]:
# Usei como base esse tutorial:
# https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/keras/regression.ipynb#scrollTo=f-OHX4DiXd8x

In [ ]:
# # # # Clear logs
# # # %rm -rf ./logs/

# # # Ativação do Tensorboard
# %conda activate pystock
# %load_ext tensorboard

# # # Reload Tensorboar
# %reload_ext tensorboard


In [ ]:
# # # Reload Tensorboar
# %reload_ext tensorboa

# Sequential model

*Dense model 16*
*Dense model 16*
*Dense model 8*
*Dense model 4*
*Epoch 350*

_MSE: 1.31_
epoch: 350

Activation: softplus
Optimizer: Adam

15/15 - 0s - loss: 0.6325 - mae: 0.5830 - mse: 0.6325
Testing set Mean Abs Error:  0.58 ibova_0

In [ ]:
import pathlib
import datetime

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
import datetime

from tensorflow import keras
from tensorflow.keras import layers

from keras.callbacks import ModelCheckpoint
import keras_tuner as kt 

# Lendo Dfs

In [ ]:
# 60%
PATH_TREINO = '../Data/3_Gold/Treino_all_stocks.csv'
DF_TREINO = pd.read_csv(PATH_TREINO, sep=",")
train_dataset = DF_TREINO
train_dataset = train_dataset.drop(['oil_5','usd_5','abev_5','jbs_5','petr_5','vale_5',], axis=1)
train_labels = train_dataset.pop('ibova_5')

# 20%
PATH_VALIDACAO = '../Data/3_Gold/Validacao_all_stocks.csv'
DF_VALIDACAO = pd.read_csv(PATH_VALIDACAO, sep=",")
valid_dataset = DF_VALIDACAO
valid_dataset = valid_dataset.drop(['ibova_5','oil_5','usd_5','abev_5','jbs_5','petr_5','vale_5',],axis=1)
# valid_labels = valid_dataset.pop('ibova_5') 

# 20%
PATH_TESTE = '../Data/3_Gold/Teste_all_stocks.csv'
DF_TESTE = pd.read_csv(PATH_TESTE, sep=",")
test_dataset = DF_TESTE
test_dataset = test_dataset.drop(['oil_5','usd_5','abev_5','jbs_5','petr_5','vale_5',],axis=1)
test_labels = test_dataset.pop('ibova_5')


In [ ]:
# Tentando criar um conjunti de validação pra input no modelo 

# val_dataset = tf.data.Dataset.from_tensor_slices(('ibova_0', '*')) 

In [ ]:
valid_dataset.ibova_0

# Pesquisar:
# 	varias colunas to series em pandas  

In [ ]:
# index,
# ibova_0,ibova_1,ibova_2,ibova_3,ibova_4,ibova_5,
# oil_0,oil_1,oil_2,oil_3,oil_4,oil_5,usd_0,
# usd_1,usd_2,usd_3,usd_4,usd_5,
# abev_0,abev_1,abev_2,abev_3,abev_4,abev_5,
# jbs_0,jbs_1,jbs_2,jbs_3,jbs_4,jbs_5,
# petr_0,petr_1,petr_2,petr_3,petr_4,petr_5,
# vale_0,vale_1,vale_2,vale_3,vale_4,vale_5

# Inspecionando os dados

In [ ]:
# sns.pairplot(train_dataset[['index','ibova_0','ibova_1','ibova_2','ibova_3','ibova_4','oil_0','oil_1','oil_2','oil_3','oil_4','usd_0','usd_1','usd_2','usd_3','usd_4']], diag_kind="kde")
# sns.pairplot(train_dataset[['index','ibova_0','oil_0','usd_0']], diag_kind="kde")
# sns.pairplot(train_dataset[['ibova_1','oil_1','usd_1']], diag_kind="kde")

train_stats = train_dataset.describe()
# train_stats

# Definindo o modelo

#TODO:

Definir um range para a definição dos hyper parametros do modelo

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection
	hyper parameter optimizers

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

A biblioteca vai encontrar os melhores parametros no conjunto de validacao 

https://www.tensorflow.org/tutorials/keras/keras_tuner

In [ ]:
def model_builder(hp):
  hp_unit_1 = hp.Int('unit_1', min_value=16, max_value=64, step=1)
  hp_unit_2 = hp.Int('unit_2', min_value=4, max_value=64, step=1)
  hp_unit_3 = hp.Int('unit_3', min_value=4, max_value=64, step=1)
  hp_unit_4 = hp.Int('unit_4', min_value=4, max_value=64, step=1)
  activationL = ["relu","sigmoid","softplus","softsign","tanh","selu","elu","exponential","relu",]
  activation_choice = hp.Choice('activation', values=activationL)
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model = keras.Sequential([
    layers.Dense(units=hp_unit_1, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(units=hp_unit_2, activation='relu'),
    layers.Dense(units=hp_unit_3, activation='relu'),
    layers.Dense(units=hp_unit_4, activation='relu'),
    layers.Dense(units=1, activation='linear')
  ])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss='mse',
                metrics=['mae', 'mse'])

  
  return model


# rmse pra avaliação, nao para loss


In [ ]:
# model.summary()

# Instanciando Tuner

https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/keras_tuner.ipynb#scrollTo=McO82AXOuxXh

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_mse',
                     max_epochs=10,
                     factor=3,
                     directory='logs',
                     project_name='hyper_parameters')

In [ ]:
# early_stop = keras.callbacks.EarlyStopping(
#   monitor='val_mse', patience=50, mode='min' ,restore_best_weights=True)
format_data = "%y_%m_%d__%H_%M_%S"
save_time = datetime.datetime.strftime(datetime.datetime.now(), format_data)

EPOCHS = 500

bst_model_path = f'logs/MLP_model/MLP_model_{save_time}.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)

history = tuner.search(train_dataset, train_labels, epochs=EPOCHS, validation_split=0.2, verbose=0, callbacks=([model_checkpoint]))

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=20)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('unit_1')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")
#TODO: Printar todas as selecoes

In [ ]:
print(best_hps.get('unit_1'))
print(best_hps.get('unit_2'))
print(best_hps.get('unit_3'))
print(best_hps.get('unit_4'))
print(best_hps.get('learning_rate'))
print(best_hps.get('activation'))

# Selecionando a melhor epoca do modelo

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs


model = tuner.hypermodel.build(best_hps)
history = model.fit(train_dataset, train_labels, epochs=EPOCHS, validation_split=0.2, verbose=1, callbacks=([model_checkpoint]))

# Select the best epoch,
val_mse_per_epoch = history.history['val_mse']
best_epoch = val_mse_per_epoch.index(min(val_mse_per_epoch)) + 1

print('Best epoch: %d' % (best_epoch,))


# Treinando modelo

In [ ]:
# hypermodel = tuner.hypermodel.build(best_hps)
# history = hypermodel.fit(train_dataset, train_labels, epochs=best_epoch, validation_split=0.2, verbose=1)

# # ja pega o modelo na melhor epoca 

In [ ]:
eval_result = hypermodel.evaluate(test_dataset, test_labels)
print("[test loss, test accuracy]:", eval_result)

# Avaliando o modelo

In [ ]:
# print(f'train_dataset: {train_dataset.columns()} \n')
# print(f'valid_dataset: {valid_dataset.columns()} \n')
# print(f'test_dataset: {test_dataset.columns()} \n')

In [ ]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [ ]:
print(f'Min MSE: {hist.mse.min()}')
print(f'Min Val_MSE: {hist.val_mse.min()}')

In [ ]:
def plot_history(history):
  hist = pd.DataFrame(history.history)
  hist['epoch'] = history.epoch

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [ibova_0]')
  plt.plot(hist['epoch'], hist['mae'],label='Train Error')
  plt.plot(hist['epoch'], hist['val_mae'],label = 'Val Error')
  plt.ylim([0,5])
  plt.legend()

  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Square Error [$ibova_0^2$]')
  plt.plot(hist['epoch'], hist['mse'],label='Train Error')
  plt.plot(hist['epoch'], hist['val_mse'],label = 'Val Error')
  plt.ylim([0,20])
  plt.legend()
  plt.show()


plot_history(history)

In [ ]:
model = build_model()

# O parâmetro patience é o quantidade de epochs para checar as melhoras
early_stop = keras.callbacks.EarlyStopping(
  monitor='val_mse', patience=25, mode='min' ,restore_best_weights=True)

history = model.fit(train_dataset, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=1, callbacks=[early_stop, tensorboard_callback])

# Mudar os dados de validação

plot_history(history)

In [ ]:
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print(f"Testing set Mean Abs Error: {mse:5.2f} ibova_0")

# Realizando as previsões

In [ ]:
test_predictions = model.predict(test_dataset).flatten()

# Avaliando as previsões:

In [ ]:
def show_test_labels():
	# Imprimindo valores reais
	plt.plot(test_labels, color ='r', label='ibova_REAL')
	plt.xlabel("Dates")
	plt.ylabel("Variation ROC")
	plt.title("ibova_REAL")
	plt.legend()

def show_model_predictions():
	# Imprimindo previsoes
	plt.plot(test_predictions, color ='g', label='predictions_MODEL')
	plt.xlabel("Dates")
	plt.ylabel("Variation ROC")
	plt.title("predictions_MODEL")
	plt.legend()

def show_compare_graph():
	# Predictt X Real values
	plt.plot(test_labels, color ='r', label='ibova_REAL')
	plt.plot(test_predictions, color ='g', label='predictions_MODEL')
	plt.xlabel("Dates")
	plt.ylabel("Variation ROC")
	plt.title("Predict X Real values")
	plt.legend()
	plt.show()

def show_true_predict_values():
	plt.figure(figsize=(24,4))
	plt.scatter(test_labels, test_predictions)
	plt.xlabel('True Values [ibova_0]')
	plt.ylabel('Predictions [ibova_0]')
	plt.axis('equal')
	plt.axis('square')
	plt.xlim([0,plt.xlim()[1]])
	plt.ylim([0,plt.ylim()[1]])
	_ = plt.plot([-100, 100], [-100, 100])

In [ ]:
show_compare_graph()

In [ ]:
show_model_predictions()

In [ ]:
# Analisando Medias:

print(f'Massa de predição: {test_predictions.mean()}')
print(f'Massa inicial: {ibova_test.mean()}')
print(f'Diferenças das médias: {ibova_test.mean() - test_predictions.mean()}')

# Add o RM_MSE medio

In [ ]:
# test_dataset